In [3]:
import ugradio
import numpy as np
import matplotlib as mpl
from matplotlib import pyplot as plt
import astropy
import math as m
import glob
import os


# import mpl_toolkits.axes_grid1 as axgrid
# from mpl_toolkits.axes_grid1 import make_axes_locatable
# import scipy

%matplotlib inline
plt.rcParams['mathtext.fontset'], plt.rcParams['font.family'] = 'stix', 'STIXGeneral'
plt.rcParams.update({'font.size': 17})

## Constants

In [99]:
### Interferometer constants ###
T_NOISE = 90 # [K]
LO, RF = 1270, 1420.4058 # [MHz]
FREQ_RES = 8192 

LT_LAT, LT_LON, LT_ALT = ugradio.leo.lat, ugradio.leo.lon, ugradio.leo.alt


### Physical constants ###
c = 3e5 # [km/s]


### Coordinate constants ###
l_min, l_max = 105, 160 # [degrees]
b_min, b_max = 15, 50 # [degrees]

δb = 2
δl = lambda b: 2/np.cos(m.radians(b)) # take in b in degrees

Δl = l_max - l_min
Δb = b_max - b_min


## Utility Functions

In [100]:
def calc_avg_power_fft(data):
    fft = np.fft.fft(data)
    pow_data = np.abs(fft)**2
    avg_data = np.mean(pow_data, axis=1)
    return np.fft.fftshift(avg_data)

def calc_avg(data):
    return np.mean(data, axis=1)

def calc_median_power_fft(data):
    fft = np.fft.fft(data)
    pow_data = np.abs(fft)**2
    avg_data = np.median(pow_data, axis=1)
    return np.fft.fftshift(avg_data)

def calc_freq(samples, frequency):
    return np.fft.fftshift(np.fft.fftfreq(samples, d=1/frequency))

def calc_gain(main_data, on_data, off_data):
    
    """
    A simple function to compute the gain for a given coordinate.
    """    
    
#     T_NOISE = 90 # Kelvin
    G = (T_NOISE/(np.sum(on_data-off_data))) * np.sum(off_data) #calc Gain
    
    return G

def calc_velocities(main_ras, main_decs, main_jds):
    
    """
    A simple function to calculate the velocities w.r.t the LSR.
    """
    
#     FREQ_RES = 8192 
#     LO = 1270 #Mhz
#     RF = 1420.4058 #Mhz
#     c = 3e5 #km/s
    freqs = np.linspace(144,156,FREQ_RES) 
    doppler_velocity = (((freqs+LO)-RF)/RF)*c #doppler velocities -- km/s 
    
#     if main_file is None:
#         correction = 0
    velocity_correction = ugradio.doppler.get_projected_velocity(ra=main_ras, 
                                                                 dec=main_decs,
                                                                 jd=main_jds,
                                                                 obs_lat=LT_LAT,
                                                                 obs_lon=LT_LON,
                                                                 obs_alt=LT_ALT)/1e3 # dividing by 1e3 converts to km/s
 
    dopp_vels = []
    for i in range(len(velocity_correction)):
        dopp_vel_lb = doppler_velocity + velocity_correction[i].value
        dopp_vels.append(dopp_vel_lb)
    return dopp_vels


def T_0(main_data, on_data, off_data, main_ras, main_decs, main_jds):
#     T_NOISE = 90 # Kelvin
    
#     G = calc_gain(main_data,on_data,off_data)
#     max_value = np.max((main_data/off_data))
#     max_index = np.argmax((main_data/off_data))
#     T = G * (max_value - 1)
#     return T

#     T_NOISE = 90 # Kelvin
    
    G = calc_gain(main_data, on_data, off_data)
    max_value = np.max((main_data/off_data))
    max_index = np.argmax((main_data/off_data))
    velocities = calc_velocities(main_ras, main_decs, main_jds)
    
    
    fitting_indices = np.where((velocities > -70) & (velocities < 50)) # we zoom in to ignore the ripples
    G = calc_gain(main_data, on_data, off_data)
    
    median = np.median(main_data[fitting_indices]/off_data[fitting_indices]) 
    
    y_data_fit = (main_data/off_data)*G - G*median
    y_data_fit = y_data_fit[fitting_indices]
    x_data_fit = velocities
    x_data_fit = x_data_fit[fitting_indices]
    max_value = np.max(y_data_fit[np.where(x_data_fit<40)])
    max_index = np.argmax(y_data_fit[np.where(x_data_fit<40)])
    T = max_value

    return T


def V_0(averaged_data, averaged_noise, averaged_no_noise):
    ind_1=np.where(velocities>-50)[0][0]# Get indices for median
    ind_2=np.where(velocities<50)[0][-1]# Get indices for median
    G=(90/(np.sum(averaged_noise-averaged_no_noise)))*np.sum(averaged_no_noise)# Get Gain
    med=np.median((averaged_data[ind_1:ind_2]/averaged_no_noise[ind_1:ind_2]))# Get median
    y_data_fit = (averaged_data/averaged_no_noise)*G-G*med
    y_data_fit = y_data_fit[ind_1:ind_2]
    x_data_fit = velocities + correction
    x_data_fit = x_data_fit[ind_1:ind_2]
    plt.plot(x_data_fit,y_data_fit)
    fit_params_on1 = ugradio.gauss.gaussfit(x_data_fit, y_data_fit, amp=50, avg=20, sig=25)#get fit
    chisq_r_s = []
    for i in range(1):
        ys = (averaged_data/averaged_no_noise)*G-med*G
        xs = velocities + correction
        chisq_r_s.append(np.sum(np.abs(y_data_fit - ugradio.gauss.gaussval(x_data_fit, **fit_params_on1))**2) / (y_data_fit.size - 3) / np.std(y_data_fit[:190])**2)
    return (fit_params_on1['avg'][0])

## Loading in Data

In [34]:
# CODE TO LOAD SPECTRA NUMPY FILES IN A DICT

data_dict = {} # dict where key is coordinate (l, b)) and value is [main, noise_on, noise_off, [ra, dec, jd]]
                # where main, noise_on, noise_off are all numpy arrays of the spectra
for filename in os.listdir(os.getcwd() + '/data/new_celestial_data/celestial_data'):
    if filename == ".ipynb_checkpoints" or filename == '.DS_Store':
        continue
    split_name = filename.split("_")

    if split_name[0] == 'off':
        fits_type = 2 # off
    elif split_name[0] == 'on':
        fits_type = 1 # on
    else:
        fits_type = 0 # main

    #l, b, ra, dec, jd
    _, l, b, ra, dec, jd, _ = split_name
    l, b, ra, dec, jd = float(l), float(b), float(ra), float(dec), float(jd) # convert strings to floats

    curr_data = np.load('data/new_celestial_data/celestial_data/' + filename) # load the spectra from file
    curr_data = np.mean(curr_data, axis=1) # keep the average in the dict
    
    if not data_dict.get((l, b)):
        data_dict[(l, b)] = [[], [], [], ra, dec, jd]      
    data_dict[(l, b)][fits_type] = curr_data
    

In [83]:
# RUN THIS CELL TO LOOP THROUGH ALL SPECTRA 
# SAVES ls, bs, ras, decs, jds, specs (main), and Ts which are ordered in the same order

ls = []
bs = []
ras, decs, jds = [], [], []
specs = []
Ts = []

for coord in data_dict.keys():
    l, b = coord
    main, noise_on, noise_off, ra, dec, jd = data_dict.get(coord)
    
    if len(main) == 0 or len(noise_on) == 0 or len(noise_off) == 0:
        print(coord)
        continue
#     temperature = T_0(main, noise_on, noise_off)
    
    ls.append(l)
    bs.append(b)
    specs.append(main)
#     Ts.append(temperature)
    ras.append(ra)
    decs.append(dec)
    jds.append(jd)

print('Length of each quantity array:', len(ls), len(bs), len(ras), len(bs), len(jds))

(158.8716594064493, 27.0)
Length of each quantity array: 400 400 400 400 400


#### Now we have lists of ls, bs, ras, decs, and JDs that are sorted and alligned. We shouldn't have to worry about sorting anything anymore or worry about allignment and misplacements.

In [101]:
velocity_correction = ugradio.doppler.get_projected_velocity(ra=ras, 
                                                                 dec=decs,
                                                                 jd=jds,
                                                                 obs_lat=LT_LAT,
                                                                 obs_lon=LT_LON,
                                                                 obs_alt=LT_ALT)/1e3

freqs = np.linspace(144,156,FREQ_RES) 
doppler_velocity = (((freqs+LO)-RF)/RF)*c #doppler velocities -- km/s 


In [103]:
# velocity_correction.value
# len(doppler_velocity)
doppler_velocity

array([-1352.95138896, -1352.64196557, -1352.33254219, ...,
        1180.91673974,  1181.22616313,  1181.53558652])

## Building

In [50]:
# bs = []
# for i in range(50): # 50 is an arbitrarily chosen number
#     b = b_min + i*δb
#     if b<b_max:
#         bs.append(b)
# bs = np.array(bs)

# # ls = []
# # for b in bs:
# #     for i in range(50):
# #         l = l_min + i*δl(b)
# #         if l<l_max:
# #             ls.append(l)
# # ls = np.array(ls)

# ls = []
# for b in bs:
#     ls.append(np.array([l_min + i*δl(b) 
#                         for i in range(50) if 
#                         (l_min + i*δl(b)) < l_max]))


        
# coords = []
# for i in range(len(bs)):
#     for j in range(len(ls[i])):
#         coord = [ls[i][j], bs[i]]
#         coords.append(coord)
# coords = np.array(coords)

# coord_names = []
# for coord in coords:
#     coord_name = f"{coord[0]:0.3f}_{coord[1]:0.3f}"
#     coord_names.append(coord_name)


## Functions of use

In [225]:
# def calc_gain(on_data, off_data):
    
#     """
#     Calculate the gain of each coordinate in the given data set.
#     """
#     gains = []
#     for i in range(len(off_data)):
#             gain = (T_NOISE*np.sum(off_data[i]))/np.sum(on_data[i]-off_data[i])
#             gains.append(gain)
#     return gains


# def calc_velocity(ras, decs, jds):
#     """
#     Calculate the velocities of each coordinate in the given data set.
#     """
    
#     freqs_range = np.linspace(144, 156, FREQ_RES)
#     doppler_velocity = (c*((freqs_range+LO)-RF))/RF
    
#     velocity_correction = ugradio.doppler.get_projected_velocity(ra=ras, 
#                                                                  dec=decs,
#                                                                  jd=jds,
#                                                                  obs_lat=LT_LAT,
#                                                                  obs_lon=LT_LON,
#                                                                  obs_alt=LT_ALT)
 
#     dopp_vels = []
#     for i in range(len(velocity_correction)):
#         dopp_vel_lb = doppler_velocity + velocity_correction[i].value
#         dopp_vels.append(dopp_vel_lb)
#     return dopp_vels


# def calc_peak_max(main_data, off_data, on_data):
#     """
#     Calibrates to convert to max temperature [K] at each point. 
#     Also computes corresponding velocity [km/s].
#     """
#     main_data = np.mean()

In [75]:
# ugradio.doppler.get_projected_velocity??

In [106]:
# freqs_range = np.linspace(144, 156, FREQ_RES)
# len(freqs_range), len(main_decs)

In [138]:
# velocity_correction = ugradio.doppler.get_projected_velocity(ra=main_ras, 
#                                                                  dec=main_decs,
#                                                                  jd=main_jds,
#                                                                  obs_lat=LT_LAT,
#                                                                  obs_lon=LT_LON,
#                                                                  obs_alt=LT_ALT)

In [105]:
# len(calc_gain(on_data, off_data))

In [212]:
# for i in range(len(on_data)):
#     print(i)

In [104]:
# for i in len(on_data):
#     print(i)